# 3DeeCellTracker Demo: Train 3D U-Net
This notebook shows how to use 3DeeCellTracker to train a 3D U-Net.
The demo data can be found in the "unet_training" folder downloaded from https://osf.io/dt76c/

The basic procedures:

- A. Import packages
- B. Initialize the parameters for training
- C. Load the train / validation datasets
- D. Preprocess the datasets
- E. Train the 3D U-Net
- F. Select the best weights and save the model

Please run following codes according to the instructions

## A. Import packages

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import warnings
warnings.filterwarnings('ignore')
from CellTracker.unet3d import TrainingUNet3D, unet3_a
%matplotlib inline

## B. Initialize the parameters for training
- noise_level: This value should be roughly the intensity of the background pixels in the raw image. It is used in
normalization to enhance the cells with weak intensity while ignore the background noises. If the two images (train and
validation) have different noise level, choose a value somehow between them.

- folder_path: This path is used to create a folder (if not exist) to store the data and model files. We recommend users
to set it as “./xxxx” to create a folder named xxxx under the same directory containing the notebook file.

- model: This should be a predefined 3D U-Net model. The simplest way is to use the default value unet3_a(). Advanced
users can select other predefined model such as unet3_b(), unet3_c(), or a model defined by themselves (need to modify
the “Import packages” cell to import the model).

In [ ]:
trainer = TrainingUNet3D(noise_level=100, folder_path=os.path.abspath("./unet_01"), model=unet3_a())

## C. Load the train / validation datasets

After running B, the program automatically generated several folders under the folder_path. Users should prepare the
training data and validation data (see examples in the demo data) and store them into the “train_image”(raw 2D image
sequence for training data), “train_label”(2D annotation of cell/non-cell regions for training data), “valid_image” and
“valid_label” (for validation data), respectively.


Run the code cell. The program will load and draw the images/annotations of the training and validation dataset by max-projection.

In [ ]:
trainer.load_dataset()
trainer.draw_dataset()

## D. Preprocess the datasets
**D1. Show the normalized images (max projection)**

Normalize the images and show the normalized images.
If the normalization looks poor, e.g. the background look too bright, or the intensity of weak cells are not enhanced.
Please go back to step B to modify the parameter “noise_level” and run all codes again.

In [ ]:
trainer.preprocess()
trainer.draw_norm_dataset()

**D2. Show the divided images (part)**

Divide the images into multiple sub-images (used as the input of 3D U-Net) and show a part of these sub-images.

In [ ]:
trainer.draw_divided_train_data()

## E. Train the 3D U-Net
Start to train the 3D U-Net. During the training, the program will draw the updated prediction of cell regions if the
loss on validation data is reduced.

By default, the program will train for 100 epochs. Users can manually stop the training by pressing Ctrl+C if the
val_loss no longer decreases anymore.

In [ ]:
trainer.train()

## F. Select the best weights and save the model
After training finished or manually stopped. Users should choose the best step which generate the best prediction of
cell regions. Usually this should be the one with the lowest val_loss, but uses can select other step instead.

The program will store the model with the chosen weights into the “models” folder with the name “unet3_pretrained.h5”

In [ ]:
trainer.select_weights(step=97)